In [1]:
import operator
import numpy as np
from math import exp
from numpy import random

class SimulatedAnnealing():
    def __init__(self, minimize, T_min, T_max, init,
                 neighbour, energy, accept, cooling):
        self.compare = operator.lt if minimize else operator.gt
        self.T_min = T_min
        self.neighbour = neighbour
        self.energy = energy
        self.accept = accept
        self.cooling = cooling
        
        self.T = T_max
        self.best = init()
        self.step = 0

    def iteration(self):
        next = self.neighbour(self.T, self.best)
        deltaE = self.energy(next) - self.energy(self.best)
        if self.compare(deltaE, 0):
            self.best = next
        elif random.random() < self.accept(self.T, deltaE):
            self.best = next

    def cool(self):
        self.T = self.cooling(self.T, self.best)

    def execute(self):
        while self.T > self.T_min:
            self.iteration()
            self.cool()
            self.log()
            self.step += 1
        print('\nf(' + str(self.best) + ') = ' + str(self.energy(self.best)))
        return self.best
    
    def log(self):
        if(self.step % 100 == 0):
            print(str(self.step) + " T="+str(self.T) +
                  "\tf(" + str(self.best) + ") = " + str(self.energy(self.best)))

In [2]:
def rosenbrock(x, y, a=1, b=100):
    return (a - x)**2 + b*(y - x**2)**2

t_min = 1e-5
t_max = 1

def init():
    return np.array([0.1, 1.4])

def neighbour(T, x):
    #return x + random.uniform(-T, T, x.shape)
    return random.normal(x, max(T, 0.1), x.shape)

def energy(vector):
    return rosenbrock(*vector)

def accept(T, deltaE, k=0.1):
    return exp(-(deltaE)/k/T)

def cooling(T, best):
    return T*0.99

rosenbrock_min = SimulatedAnnealing(True, t_min, t_max, init,
                                    neighbour, energy, accept, cooling).execute()

0 T=0.99	f([ 0.1  1.4]) = 194.02
100 T=0.36237201786049694	f([ 1.45027075  2.14129947]) = 0.347251799283
200 T=0.13263987810938213	f([ 1.34198948  1.8309866 ]) = 0.207262127206
300 T=0.0485504851305729	f([ 1.2804645   1.65393784]) = 0.0992482852046
400 T=0.017771047742294682	f([ 1.20762644  1.46450077]) = 0.0468776597348
500 T=0.006504778211990459	f([ 1.19064942  1.40987116]) = 0.042392092044
600 T=0.0023809591983979563	f([ 1.17785761  1.39465283]) = 0.0369685659074
700 T=0.0008715080698656353	f([ 1.08762651  1.17878099]) = 0.0094010243823
800 T=0.00031900013925143135	f([ 0.96526271  0.93191266]) = 0.00120994017387
900 T=0.00011676436783668758	f([ 0.96526271  0.93191266]) = 0.00120994017387
1000 T=4.2739534936551264e-05	f([ 0.96526271  0.93191266]) = 0.00120994017387
1100 T=1.564405203775484e-05	f([ 0.98014312  0.9625191 ]) = 0.00073232756491

f([ 0.98014312  0.9625191 ]) = 0.00073232756491


In [3]:
# 0-1 Knapsack Problem data taken from
# https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html
# optimal profit = 1458
# optimal selection = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

DIM = 15
W = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
V = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
C = 750

def kp_init():
    return np.random.randint(2, size=DIM)

def kp_neighbour(T, knapsack):
    new_knapsack = np.copy(knapsack)
    new_knapsack[random.randint(DIM)] ^= 1
    while kp_cost(new_knapsack) == 0:
        new_knapsack[random.randint(DIM)] ^= 1
    return new_knapsack

def kp_cost(knapsack):
    total_weight = sum([W[i] for i, chosen in enumerate(knapsack) if chosen])
    total_value = sum([V[i] for i, chosen in enumerate(knapsack) if chosen])
    return 0 if total_weight > C else total_value

def kp_accept(T, deltaE, k=0.1):
    return exp(deltaE / k / T)

kp_max = SimulatedAnnealing(False, t_min, t_max, kp_init,
                            kp_neighbour, kp_cost, kp_accept, cooling).execute()

0 T=0.99	f([0 1 0 1 1 0 0 0 1 0 0 1 1 0 1]) = 1312
100 T=0.36237201786049694	f([1 0 1 1 1 0 0 1 0 0 0 1 1 1 0]) = 1438
200 T=0.13263987810938213	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
300 T=0.0485504851305729	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
400 T=0.017771047742294682	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
500 T=0.006504778211990459	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
600 T=0.0023809591983979563	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
700 T=0.0008715080698656353	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
800 T=0.00031900013925143135	f([1 1 1 0 0 0 1 1 0 0 1 0 1 0 1]) = 1451
900 T=0.00011676436783668758	f([0 1 1 0 1 0 1 1 1 0 0 0 1 0 1]) = 1454
1000 T=4.2739534936551264e-05	f([0 1 1 0 1 0 1 1 1 0 0 0 1 0 1]) = 1454
1100 T=1.564405203775484e-05	f([0 1 1 0 1 0 1 1 1 0 0 0 1 0 1]) = 1454

f([0 1 1 0 1 0 1 1 1 0 0 0 1 0 1]) = 1454


In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# %matplotlib nbagg

# animation of Rosenbrock minimization
sa = SimulatedAnnealing(True, t_min, t_max, init, neighbour, energy, accept, cooling)
fig = plt.figure()
xmin, xmax = -2.0, 2.0
ymin, ymax = -1.0, 3.0
x1 = np.linspace(xmin, xmax, 100)
x2 = np.linspace(ymin, ymax, 100)
Xc, Yc = np.meshgrid(x1, x2)
Zc = rosenbrock(Xc, Yc)
ax = fig.add_subplot(111, aspect='equal', autoscale_on=False, xlim=(xmin, xmax), ylim=(ymin, ymax))
CS = ax.contour(Xc, Yc, Zc, levels=[0, 0.1, 1, 2, 10, 20, 100])
ax.clabel(CS, inline=1, fmt='%1.1f', fontsize=14)
ax.set_title('Non-Convex Function')
line, = ax.plot([], [], 'o-', lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
energy_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)

def init_animation():
    line.set_data([], [])
    time_text.set_text('')
    energy_text.set_text('')
    return line, time_text, energy_text

xs = []
ys = []
def animate(step):
    global sa, xs, ys
    sa.iteration()
    sa.cool()
    xs.append(sa.best[0])
    ys.append(sa.best[1])
    line.set_data(xs, ys)
    time_text.set_text('time = %.1f' % step)
    energy_text.set_text('temperature = %.3f J' % sa.T)
    return line, time_text, energy_text

ani = animation.FuncAnimation(fig, animate, frames=25, interval=25, init_func=init_animation)
plt.show()